<a href="https://colab.research.google.com/github/am4279/Toxic_Comments_Kaggle_Competition/blob/main/BigRU_Toxic_Comments.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [3]:
!python --version

Python 3.7.11


In [4]:
!pip install ktrain

     |████████████████████████████████| 25.3 MB 1.2 MB/s 
     |████████████████████████████████| 6.8 MB 57.1 MB/s 
     |████████████████████████████████| 981 kB 40.5 MB/s 
     |████████████████████████████████| 263 kB 46.9 MB/s 
     |████████████████████████████████| 1.9 MB 42.9 MB/s 
     |████████████████████████████████| 1.2 MB 58.1 MB/s 
     |████████████████████████████████| 468 kB 50.7 MB/s 
     |████████████████████████████████| 895 kB 41.0 MB/s 
     |████████████████████████████████| 3.3 MB 52.5 MB/s 
  Created wheel for ktrain: filename=ktrain-0.27.2-py3-none-any.whl size=25283088 sha256=6bae2770cba63e8da11de41d337e32b0a9373746105636b2ca0f0cd07a9bc4c0
  Stored in directory: /root/.cache/pip/wheels/88/be/4a/971c83a380a40f12e877f643ca1b94dc65f528f94c88dbcff7
  Created wheel for seqeval: filename=seqeval-0.0.19-py3-none-any.whl size=9929 sha256=7ef95a8ac42a63f69bb3d8cc19888d339e9deebdab2f3591f0a50ae056275ae1
  Stored in directory: /root/.cache/pip/wheels/f5/ac/f1/4e13d7aff

In [ ]:
i =0.000000000000000000000000000000000000000000000000000000000000000000000000000000000000000003

while i>0.000000000000000000000000000000000000000000000000000000000000000000000000000000000000000002:
  i= i+0.000000000000000000000000000000000000000000000000000000000000000000000000000000000000000001

In [37]:
#i

In [6]:
# from google.colab import drive
# drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [12]:
import ktrain
from ktrain import text


#DATA_PATH = '/Users/andrewmccurdy/Dropbox/University of Chicago/Summer_2021/Adv_AI/jigsaw-toxic-comment-classification-challenge/train.csv'

DATA_PATH = '/content/drive/MyDrive/toxic_train.csv'
NUM_WORDS = 50000
MAXLEN = 150
(x_train, y_train), (x_test, y_test), preproc = text.texts_from_csv(DATA_PATH, 'comment_text',
                      label_columns = ["toxic", "severe_toxic", "obscene", "threat", "insult", "identity_hate"],
                      val_filepath=None, # if None, 10% of data will be used for validation
                      max_features=NUM_WORDS, maxlen=MAXLEN, ngram_range=1)

detected encoding: utf-8 (if wrong, set manually)
['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate']
       toxic  severe_toxic  obscene  threat  insult  identity_hate
71522      0             0        0       0       0              0
95665      0             0        0       0       0              0
20755      0             0        0       0       0              0
62094      0             0        0       0       0              0
80238      0             0        0       0       0              0
['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate']
        toxic  severe_toxic  obscene  threat  insult  identity_hate
133734      0             0        0       0       0              0
126488      0             0        0       0       0              0
102966      0             0        0       0       0              0
137741      1             0        1       0       1              0
123615      0             0        0       0       0        

In [13]:
text.print_text_classifiers()


fasttext: a fastText-like model [http://arxiv.org/pdf/1607.01759.pdf]
logreg: logistic regression using a trainable Embedding layer
nbsvm: NBSVM model [http://www.aclweb.org/anthology/P12-2018]
bigru: Bidirectional GRU with pretrained fasttext word vectors [https://fasttext.cc/docs/en/crawl-vectors.html]
standard_gru: simple 2-layer GRU with randomly initialized embeddings
bert: Bidirectional Encoder Representations from Transformers (BERT) from keras_bert [https://arxiv.org/abs/1810.04805]
distilbert: distilled, smaller, and faster BERT from Hugging Face transformers [https://arxiv.org/abs/1910.01108]


In [14]:
model = text.text_classifier('bigru', (x_train, y_train), preproc=preproc)

learner = ktrain.get_learner(model, train_data=(x_train, y_train), val_data=(x_test, y_test))

Is Multi-Label? True
compiling word ID features...
maxlen is 150
word vectors will be loaded from: https://dl.fbaipublicfiles.com/fasttext/vectors-crawl/cc.en.300.vec.gz
processing pretrained word vectors...
downloading pretrained word vectors to /root/ktrain_data ...
[██████████████████████████████████████████████████]
extracting pretrained word vectors...
done.

cleanup downloaded zip...
done.

loading pretrained word vectors...this may take a few moments...


done.


In [16]:
# learner.lr_find()
# learner.lr_plot()

In [19]:
# define a custom callback for ROC-AUC
from tensorflow.keras.callbacks import Callback
from sklearn.metrics import roc_auc_score
class RocAucEvaluation(Callback):
    def __init__(self, validation_data=(), interval=1):
        super(Callback, self).__init__()

        self.interval = interval
        self.X_val, self.y_val = validation_data

    def on_epoch_end(self, epoch, logs={}):
        if epoch % self.interval == 0:
            y_pred = self.model.predict(self.X_val, verbose=0)
            score = roc_auc_score(self.y_val, y_pred)
            print("\n ROC-AUC - epoch: %d - score: %.6f \n" % (epoch+1, score))
RocAuc = RocAucEvaluation(validation_data=(x_test, y_test), interval=1)

In [20]:
# train
learner.autofit(0.001, 3, callbacks=[RocAuc])



begin training using triangular learning rate policy with max lr of 0.001...
Epoch 1/3
4488/4488 [==============================] - 878s 195ms/step - loss: 0.0696 - accuracy: 0.9495 - val_loss: 0.0440 - val_accuracy: 0.9937

 ROC-AUC - epoch: 1 - score: 0.980043 

Epoch 2/3
4488/4488 [==============================] - 869s 194ms/step - loss: 0.0404 - accuracy: 0.9802 - val_loss: 0.0418 - val_accuracy: 0.9900

 ROC-AUC - epoch: 2 - score: 0.985360 

Epoch 3/3
4488/4488 [==============================] - 877s 195ms/step - loss: 0.0347 - accuracy: 0.9461 - val_loss: 0.0423 - val_accuracy: 0.9599

 ROC-AUC - epoch: 3 - score: 0.986196 



In [21]:
from sklearn.metrics import roc_auc_score
y_pred = learner.model.predict(x_test, verbose=0)
score = roc_auc_score(y_test, y_pred)
print("\n ROC-AUC score: %.6f \n" % (score))


 ROC-AUC score: 0.986196 



In [22]:
y_pred

array([[3.9053557e-05, 4.6440291e-06, 2.3921595e-05, 5.6555978e-06,
        2.3364890e-05, 4.8844167e-06],
       [5.0792098e-04, 2.7477401e-05, 2.0000339e-04, 3.5041889e-05,
        1.2761354e-04, 2.1319449e-05],
       [1.4436245e-04, 5.3383828e-06, 2.9649695e-05, 1.6595641e-05,
        3.5240351e-05, 5.6212673e-05],
       ...,
       [9.5338035e-01, 3.1930745e-02, 6.1430377e-01, 1.0081589e-02,
        5.9612310e-01, 8.1845939e-02],
       [9.2756999e-01, 6.1790943e-03, 6.0969949e-02, 3.1816661e-03,
        5.3731787e-01, 4.1678947e-01],
       [7.2356728e-05, 2.7535307e-06, 2.9576069e-05, 9.3311401e-06,
        1.2698837e-05, 4.7416984e-06]], dtype=float32)

In [23]:
predictor = ktrain.get_predictor(learner.model, preproc)

In [33]:
import pandas as pd

test = pd.read_csv('/content/drive/MyDrive/toxic_test.csv',index_col=0)

# TC_TEST_DATA_PATH = '/content/tc_test.csv'
# tc_test = text.texts_from_csv(TC_TEST_DATA_PATH, 'comment_text', label_columns = [],#"toxic", "severe_toxic", "obscene", "threat", "insult", "identity_hate"],
#                       val_filepath=None, # if None, 10% of data will be used for validation
#                       max_features=NUM_WORDS, maxlen=MAXLEN, ngram_range=1)

In [34]:
#test = pd.read_csv('tc_test.csv',index_col=0)
test.head()

,comment_text
id,
00001cee341fdb12,Yo bitch Ja Rule is more succesful then you'll...
0000247867823ef7,== From RfC == \n\n The title is fine as it is...
00013b17ad220c46,""" \n\n == Sources == \n\n * Zawe Ashton on Lap..."
00017563c3f7919a,":If you have a look back at the source, the in..."
00017695ad8997eb,I don't anonymously edit articles at all.


In [27]:
test['toxic'] = '0'
test['severe_toxic'] = '0'
test['obscene'] = '0'
test['threat'] = '0'
test['insult'] = '0'
test['identity_hate'] = '0'

In [28]:
comments_test = test['comment_text'].values
detection_classes = ['toxic', 'severe_toxic', 'obscene', 'threat',
       'insult', 'identity_hate']
target_classes_test = test[detection_classes].values

In [29]:
# Max and Min Length
print(f'Maximum length of the comments {max(len(s) for s in comments_test)}')
print(f'Minimum length of the comments {min(len(s) for s in comments_test)}')

# Median Length
s = sorted(len(s) for s in comments_test)
print(f'Median length of the comments {s[len(s) // 2]}')

Maximum length of the comments 5000
Minimum length of the comments 1
Median length of the comments 169


In [ ]:
# from keras.preprocessing.text import Tokenizer

# tokenizer_test = Tokenizer(num_words=20000)
# tokenizer_test.fit_on_texts(comments_test)
# sequences_test = tokenizer.texts_to_sequences(comments_test)

In [ ]:
# word_to_index_test = tokenizer_test.word_index
# print('Found %s unique tokens.' % len(word_to_index_test))

In [ ]:
# from keras.preprocessing.sequence import pad_sequences
# data_test = pad_sequences(sequences_test, maxlen=100)
# print('Shape of data tensor:', data_test.shape)

In [ ]:
#r = model.predict(data_test)
#predictor.predict(df.data.iloc[0:NumRecs].tolist())
#r = predictor.predict(test.comment_text)

r = predictor.predict(test.comment_text.tolist())

In [ ]:
type(r)

In [ ]:
r[0]

In [ ]:
import numpy as np
y=np.array(r)
y

In [ ]:
yt = y.T
#yt[1]
rz = yt[1]

In [ ]:
dataset1 = pd.DataFrame({'toxic': rz[0, :], 'severe_toxic': rz[1, :], 'obscene': rz[2, :], 'threat': rz[3, :], 'insult': rz[4, :], 'identity_hate': rz[5, :]})

In [ ]:
#dataset = pd.DataFrame(r)


In [ ]:
dataset1

In [ ]:
dataset1['id'] = test.index
dataset1

In [ ]:
datasettt = dataset1[['id', 'toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate']]


In [ ]:
datasettt


In [ ]:
datasettt.to_csv(r'/content/drive/MyDrive/dataframe2.csv', index = False)


In [ ]:
# NumRecs = len(tc_test)

# target = df.target.iloc[0:NumRecs]
# predicted = predictor.predict(df.data.iloc[0:NumRecs].tolist())
# data = df.data.iloc[0:NumRecs]

# results = pd.DataFrame(list(zip(target, predicted, data)), 
#                columns =['target', 'predicted', 'data']) 

In [39]:
df = pd.read_csv('/content/drive/MyDrive/toxic_train.csv')

In [40]:
df

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,0000997932d777bf,Explanation\nWhy the edits made under my usern...,0,0,0,0,0,0
1,000103f0d9cfb60f,D'aww! He matches this background colour I'm s...,0,0,0,0,0,0
2,000113f07ec002fd,"Hey man, I'm really not trying to edit war. It...",0,0,0,0,0,0
3,0001b41b1c6bb37e,"""\nMore\nI can't make any real suggestions on ...",0,0,0,0,0,0
4,0001d958c54c6e35,"You, sir, are my hero. Any chance you remember...",0,0,0,0,0,0
...,...,...,...,...,...,...,...,...
159566,ffe987279560d7ff,""":::::And for the second time of asking, when ...",0,0,0,0,0,0
159567,ffea4adeee384e90,You should be ashamed of yourself \n\nThat is ...,0,0,0,0,0,0
159568,ffee36eab5c267c9,"Spitzer \n\nUmm, theres no actual article for ...",0,0,0,0,0,0
159569,fff125370e4aaaf3,And it looks like it was actually you who put ...,0,0,0,0,0,0
